# 🚀 Requête WIKIDATA

## 🔨 Construction de l'environnement nécessaire et configuration

### Installation des modules

In [ ]:
%pip install SPARQLWrapper tqdm pandas

print("⬇️ Installation terminée !")

### Configuration

In [1]:
# 🔧 CONFIGURATION ET IMPORTS
import json
import csv
import time
import re
import os
from datetime import datetime
from SPARQLWrapper import SPARQLWrapper, JSON
from tqdm import tqdm
import pandas as pd

# Configuration
WIKIDATA_ENDPOINT = "https://query.wikidata.org/sparql"
RATE_LIMIT_DELAY = 3.0 
BATCH_SIZE = 10  
MAX_RETRIES = 3  
REQUEST_TIMEOUT = 60
ENRICHMENT_BATCH_SIZE = 15
LOOP_LIMIT = 25
LOOP_OFFSET = 0

# Dossier de sortie
output_dir = "./output"
os.makedirs(output_dir, exist_ok=True)

print("🚀 CONFIGURATION TERMINÉE")
print(f"📁 Dossier de sortie: {output_dir}")
print(f"⏱️  Rate limit: {RATE_LIMIT_DELAY}s entre requêtes")
print(f"📦 Taille des batches: {BATCH_SIZE}")

🚀 CONFIGURATION TERMINÉE
📁 Dossier de sortie: ./output
⏱️  Rate limit: 3.0s entre requêtes
📦 Taille des batches: 10


In [2]:
def create_sparql_client():
    """
    Créer un client SPARQL pour interagir avec Wikidata
    :return: Instance de SPARQLWrapper configurée pour Wikidata
    """
    sparql = SPARQLWrapper(WIKIDATA_ENDPOINT)
    sparql.setReturnFormat(JSON)
    sparql.setTimeout(REQUEST_TIMEOUT)
    return sparql

def execute_sparql_query(query, max_retries=MAX_RETRIES, use_pagination=False, limit=None, max_results=None):
    """
    Exécute une requête SPARQL avec gestion des erreurs, rate limiting et pagination optionnelle
    :param query: La requête SPARQL à exécuter
    :param max_retries: Nombre maximum de tentatives en cas d'échec
    :param use_pagination: Si True, active la pagination automatique
    :param limit: Taille des pages pour la pagination (défaut: LOOP_LIMIT)
    :param max_results: Nombre maximum de résultats à récupérer (None = illimité)
    :return: Résultats de la requête ou une liste vide en cas d'échec
    """
    sparql = create_sparql_client()
    
    # Mode simple sans pagination
    if not use_pagination:
        for attempt in range(max_retries):
            try:
                sparql.setQuery(query)
                results = sparql.query().convert()
                time.sleep(RATE_LIMIT_DELAY)
                return results["results"]["bindings"]
            except Exception as e:
                print(f"⚠️  Tentative {attempt + 1}/{max_retries} échouée: {e}...")
                if attempt < max_retries - 1:
                    time.sleep(RATE_LIMIT_DELAY * (attempt + 2))
                else:
                    print(f"❌ Requête échouée après {max_retries} tentatives")
                    return []
        return []
    
    # Mode pagination
    if limit is None:
        limit = LOOP_LIMIT
    
    all_results = []
    offset = 0
    
    print(f"🔍 Début de la pagination (limit={limit})...")
    
    while True:
        # Ajouter LIMIT et OFFSET à la requête
        paginated_query = f"{query.rstrip()} LIMIT {limit} OFFSET {offset}"
        
        print(f"🔹 Requête OFFSET {offset}, LIMIT {limit}")
        
        success = False
        for attempt in range(max_retries):
            try:
                sparql.setQuery(paginated_query)
                results = sparql.query().convert()
                bindings = results["results"]["bindings"]
                
                if not bindings:
                    print("✅ Fin de la pagination - Aucun résultat supplémentaire.")
                    success = True
                    break
                
                all_results.extend(bindings)
                print(f"✅ Récupéré {len(bindings)} résultats (total: {len(all_results)})")
                
                # Vérifier la limite max_results
                if max_results and len(all_results) >= max_results:
                    print(f"🎯 Limite de {max_results} résultats atteinte.")
                    all_results = all_results[:max_results]  # Tronquer si nécessaire
                    success = True
                    break
                
                success = True
                break
                
            except Exception as e:
                print(f"⚠️  Tentative {attempt + 1}/{max_retries} échouée à l'offset {offset}: {e}")
                if attempt < max_retries - 1:
                    time.sleep(RATE_LIMIT_DELAY * (attempt + 2))
                else:
                    print(f"❌ Requête échouée après {max_retries} tentatives à l'offset {offset}")
                    return all_results  # Retourner ce qu'on a réussi à récupérer
        
        if not success or (max_results and len(all_results) >= max_results):
            break
        
        # Incrémenter l'offset pour la prochaine page
        offset += limit
        
        # Rate limiting crucial
        print(f"⏳ Attente de {RATE_LIMIT_DELAY}s...")
        time.sleep(RATE_LIMIT_DELAY)
    
    print(f"🎯 Total final récupéré : {len(all_results)} résultats.")
    return all_results

def clean_entity_id(entity_uri):
    """
    Extrait l'ID d'une entité à partir de son URI
    :param entity_uri: URI de l'entité (ex: "http://www.wikidata.org/entity/Q42'")
    :return: ID de l'entité (ex: "Q42") ou une chaîne vide si l'URI est vide
    """
    if not entity_uri:
        return ""
    return entity_uri.split("/")[-1] if "/" in entity_uri else entity_uri

def execute_batch_queries(queries, description="Requêtes", use_pagination=False):
    """
    Exécute une liste de requêtes SPARQL en batch
    :param queries: Liste de requêtes SPARQL à exécuter
    :param description: Description de la tâche pour l'affichage
    :param use_pagination: Active la pagination pour chaque requête
    :return: Liste de tous les résultats combinés
    """
    all_results = []
    for i, query in enumerate(tqdm(queries, desc=description)):
        results = execute_sparql_query(query, use_pagination=use_pagination)
        all_results.extend(results)
        if (i + 1) % BATCH_SIZE == 0:
            time.sleep(RATE_LIMIT_DELAY)
    return all_results

def execute_paginated_query(base_query, limit=None, max_results=None):
    """
    Fonction helper pour exécuter facilement une requête avec pagination
    :param base_query: Requête SPARQL de base (sans LIMIT/OFFSET)
    :param limit: Taille des pages (défaut: LOOP_LIMIT)
    :param max_results: Nombre maximum de résultats (None = illimité)
    :return: Liste de tous les résultats
    """
    return execute_sparql_query(
        base_query, 
        use_pagination=True, 
        limit=limit, 
        max_results=max_results
    )

print("""🔗 FONCTIONS CONFIGURÉES :
      - create_sparql_client: Crée un client SPARQL pour Wikidata
      - execute_sparql_query: Exécute une requête SPARQL avec gestion
        des erreurs, rate limiting et pagination optionnelle
        - clean_entity_id: Extrait l'ID d'une entité à partir de son URI
        - execute_batch_queries: Exécute une liste de requêtes SPARQL en batch
        - execute_paginated_query: Exécute une requête SPARQL avec pagination
      """)

🔗 FONCTIONS CONFIGURÉES :
      - create_sparql_client: Crée un client SPARQL pour Wikidata
      - execute_sparql_query: Exécute une requête SPARQL avec gestion
        des erreurs, rate limiting et pagination optionnelle
        - clean_entity_id: Extrait l'ID d'une entité à partir de son URI
        - execute_batch_queries: Exécute une liste de requêtes SPARQL en batch
        - execute_paginated_query: Exécute une requête SPARQL avec pagination
      


## 🚧 Construction de la requête


### Rechercher une entité par nom

In [3]:
# 🎯 IDENTIFICATION DE L'ENTITÉ

def find_aeronautics_entity():
    """
    Demande à l'utilisateur le nom de l'entité à rechercher dans Wikidata
    :return: Tuple contenant l'ID et le label de l'entité
    """
    entity_name = input("Entrez le nom de l'entité à rechercher dans wikidata, en anglais (ex: aeronautics) : ").strip()
    if not entity_name:
        print("❌ Aucun nom d'entité fourni.")
        return None, None  # ✅ Cohérent avec l'assignation

    # Choix de la requête à utiliser
    query_choice = input(
        "Quel type de requête utiliser ?\n"
        "1️⃣ Recherche d'une chaîne dans les labels\n"
        "2️⃣ Requête d'une chaîne dans les parents\n"
        "Entrez le numéro de votre choix (1 ou 2) : "
    ).strip()

    if query_choice == "1":
        query_regex = f"""
        SELECT ?item ?itemLabel
        WHERE {{
            ?item rdfs:label ?itemLabel.
            FILTER(LANG(?itemLabel) = "en").
            FILTER(CONTAINS(LCASE(?itemLabel), "{entity_name}")).
        }}
        """
    elif query_choice == "2":
        query_regex = f"""
        SELECT ?item ?itemLabel
        WHERE {{
            ?item wdt:P31*/wdt:P279* ?parent.
            ?parent rdfs:label ?parentLabel.
            FILTER(LANG(?parentLabel) = "en").
            FILTER(CONTAINS(LCASE(?parentLabel), "{entity_name}")).
            SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
        }}
        """
    else:
        print("❌ Choix invalide. Veuillez entrer 1 ou 2.")
        return None, None  # ✅ Cohérent avec l'assignation

    print(f"🔍 Recherche de l'entité principale '{entity_name}'...")
    results = execute_paginated_query(query_regex)  # Limite pour voir plusieurs résultats

    if results:
        print(f"\n📋 {len(results)} entités trouvées:")
        print("-" * 50)
        
        # Afficher les premiers résultats pour que l'utilisateur puisse voir
        for i, entity in enumerate(results[:5]):
            entity_id = clean_entity_id(entity["item"]["value"])
            entity_label = entity["itemLabel"]["value"]
            print(f"{i+1}. {entity_label} ({entity_id})")
        
        if len(results) > 5:
            print(f"... et {len(results) - 5} autres résultats")

In [4]:
def execute_and_save_query(query, filename=None, limit=None, max_results=None, description="Requête SPARQL"):
    """
    Exécute une requête SPARQL avec pagination et sauvegarde automatiquement en JSON
    :param query: Requête SPARQL à exécuter
    :param filename: Nom du fichier JSON (sans extension). Si None, génère automatiquement
    :param limit: Taille des pages pour la pagination (défaut: LOOP_LIMIT)
    :param max_results: Nombre maximum de résultats (None = illimité)
    :param description: Description pour les logs
    :return: Tuple (results, json_filepath) - Résultats et chemin du fichier JSON
    """
    print(f"🚀 EXÉCUTION: {description}")
    print("="*50)
    
    # Exécuter la requête avec pagination
    print(f"🔍 Exécution de la requête...")
    results = execute_paginated_query(query, limit=limit, max_results=max_results)
    
    if not results:
        print("❌ Aucun résultat trouvé")
        return [], None
    
    print(f"✅ {len(results)} résultats récupérés")
    
    # Générer le nom de fichier si non fourni
    if filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"query_results_{timestamp}"
    
    # Ajouter l'extension .json si elle n'est pas présente
    if not filename.endswith('.json'):
        filename += '.json'
    
    # Chemin complet du fichier
    json_filepath = os.path.join(output_dir, filename)
    
    # Créer les données à sauvegarder avec métadonnées
    data_to_save = {
        "metadata": {
            "description": description,
            "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "total_results": len(results),
            "query": query.strip(),
            "pagination_limit": limit or LOOP_LIMIT,
            "max_results": max_results
        },
        "results": results
    }
    
    # Sauvegarder en JSON
    try:
        with open(json_filepath, 'w', encoding='utf-8') as jsonfile:
            json.dump(data_to_save, jsonfile, ensure_ascii=False, indent=2)
        
        print(f"💾 Résultats sauvegardés: {filename}")
        print(f"📁 Chemin: {json_filepath}")
        
        # Afficher un aperçu des résultats
        print(f"\n📊 APERÇU DES RÉSULTATS:")
        for i, result in enumerate(results[:3]):
            print(f"🔹 Résultat {i+1}:")
            for key, value in result.items():
                if isinstance(value, dict) and 'value' in value:
                    print(f"   • {key}: {value['value'][:100]}...")
                else:
                    print(f"   • {key}: {str(value)[:100]}...")
            print()
        
        if len(results) > 3:
            print(f"... et {len(results) - 3} autres résultats")
        
        return results, json_filepath
        
    except Exception as e:
        print(f"❌ Erreur lors de la sauvegarde: {e}")
        return results, None

def execute_custom_query():
    """
    Fonction interactive pour exécuter une requête personnalisée
    """
    print("🎯 EXÉCUTION D'UNE REQUÊTE PERSONNALISÉE")
    print("="*45)
    
    # Demander la requête à l'utilisateur
    print("📝 Entrez votre requête SPARQL (tapez 'END' sur une ligne vide pour terminer):")
    query_lines = []
    while True:
        line = input()
        if line.strip().upper() == 'END':
            break
        query_lines.append(line)
    
    query = '\n'.join(query_lines)
    
    if not query.strip():
        print("❌ Requête vide")
        return None, None
    
    # Demander les paramètres optionnels
    description = input("📋 Description de la requête (optionnel): ").strip()
    if not description:
        description = "Requête personnalisée"
    
    filename = input("📁 Nom du fichier de sauvegarde (optionnel, sans extension): ").strip()
    if not filename:
        filename = None
    
    limit_input = input(f"📊 Limite par page (défaut: {LOOP_LIMIT}): ").strip()
    limit = int(limit_input) if limit_input.isdigit() else None
    
    max_results_input = input("🎯 Nombre maximum de résultats (optionnel): ").strip()
    max_results = int(max_results_input) if max_results_input.isdigit() else None
    
    # Exécuter la requête
    return execute_and_save_query(
        query=query,
        filename=filename,
        limit=limit,
        max_results=max_results,
        description=description
    )

print("✅ Fonctions d'exécution générique configurées:")
print("   • execute_and_save_query(): Exécute et sauvegarde une requête")
print("   • execute_custom_query(): Interface interactive pour requêtes personnalisées")

✅ Fonctions d'exécution générique configurées:
   • execute_and_save_query(): Exécute et sauvegarde une requête
   • execute_custom_query(): Interface interactive pour requêtes personnalisées


In [5]:
# 🎯 IDENTIFICATION DE L'ENTITÉ

def find_aeronautics_entity():
    """
    Demande à l'utilisateur le nom de l'entité à rechercher dans Wikidata
    :return: Tuple contenant l'ID et le label de l'entité
    """
    entity_name = input("Entrez le nom de l'entité à rechercher dans wikidata, en anglais (ex: aeronautics) : ").strip()
    if not entity_name:
        print("❌ Aucun nom d'entité fourni.")
        return None, None  # ✅ Cohérent avec l'assignation

    # Choix de la requête à utiliser
    query_choice = input(
        "Quel type de requête utiliser ?\n"
        "1️⃣ Recherche d'une chaîne dans les labels\n"
        "2️⃣ Requête d'une chaîne dans les parents\n"
        "Entrez le numéro de votre choix (1 ou 2) : "
    ).strip()

    if query_choice == "1":
        query_regex = f"""
        SELECT ?item ?itemLabel
        WHERE {{
            ?item rdfs:label ?itemLabel.
            FILTER(LANG(?itemLabel) = "en").
            FILTER(CONTAINS(LCASE(?itemLabel), "{entity_name}")).
        }}
        """
    elif query_choice == "2":
        query_regex = f"""
        SELECT ?item ?itemLabel
        WHERE {{
            ?item wdt:P31*/wdt:P279* ?parent.
            ?parent rdfs:label ?parentLabel.
            FILTER(LANG(?parentLabel) = "en").
            FILTER(CONTAINS(LCASE(?parentLabel), "{entity_name}")).
            SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
        }}
        """
    else:
        print("❌ Choix invalide. Veuillez entrer 1 ou 2.")
        return None, None  # ✅ Cohérent avec l'assignation

    print(f"🔍 Recherche de l'entité principale '{entity_name}'...")
    print(f"🔍 Requête sélectionnée:{query_regex}")
    return execute_and_save_query(query_regex)
find_aeronautics_entity()
    

🔍 Recherche de l'entité principale 'gallifrey'...
🔍 Requête sélectionnée:
        SELECT ?item ?itemLabel
        WHERE {
            ?item rdfs:label ?itemLabel.
            FILTER(LANG(?itemLabel) = "en").
            FILTER(CONTAINS(LCASE(?itemLabel), "gallifrey")).
        }
        
🚀 EXÉCUTION: Requête SPARQL
🔍 Exécution de la requête...
🔍 Début de la pagination (limit=25)...
🔹 Requête OFFSET 0, LIMIT 25
⚠️  Tentative 1/3 échouée à l'offset 0: The read operation timed out
⚠️  Tentative 2/3 échouée à l'offset 0: The read operation timed out
⚠️  Tentative 3/3 échouée à l'offset 0: The read operation timed out
❌ Requête échouée après 3 tentatives à l'offset 0
❌ Aucun résultat trouvé


([], None)

### Requêtes SPARQL

In [6]:
def build_aeronautics_extraction_queries():
    """Construit les requêtes d'extraction des données"""
    queries = {
        "manufacturers": """
        SELECT DISTINCT ?item 
              
                ?itemLabel
               (COALESCE(?itemDescription_fr, ?itemDescription_en, ?itemDescription_any, "") AS ?itemDescription)
               ?parent
               ?parentLabel
               (GROUP_CONCAT(DISTINCT ?synonym_fr; separator=",") AS ?synonyms_fr)
        WHERE {
          # Tous les équipements d'aviation (instances ou sous-classes ou parties)
          ?item wdt:P31*/wdt:P279*/wdt:P361*/wdt:P452*/wdt:P749* wd:Q936518 .
        
          # On cherche le parent immédiat selon différentes relations
            OPTIONAL { ?item wdt:P361 ?partOf . }
            OPTIONAL { ?item wdt:P279 ?subclassOf . }
            OPTIONAL { ?item wdt:P31 ?instanceOf . }
            OPTIONAL { ?item wdt:P452 ?secteur .}
            OPTIONAL { ?item wdt:P176 ?constructeur.}
            OPTIONAL { ?item wdt:P749 ?constructeur.}
            
            BIND(COALESCE(?partOf, ?subclassOf, ?instanceOf, ?secteur, ?constructeur) AS ?parent)
        
          # Description de l'item (fr > en > autre)
          OPTIONAL { ?item schema:description ?itemDescription_fr . FILTER(LANG(?itemDescription_fr) = "fr") }
          OPTIONAL { ?item schema:description ?itemDescription_en . FILTER(LANG(?itemDescription_en) = "en") }
          OPTIONAL { ?item schema:description ?itemDescription_any .
                     FILTER(LANG(?itemDescription_any) != "fr" && LANG(?itemDescription_any) != "en") }

          # Synonymes en français (P1709 est "synonymes exacts")
          OPTIONAL { ?item skos:altLabel ?synonym_fr . FILTER(LANG(?synonym_fr) = "fr") }
        
          SERVICE wikibase:label {
            bd:serviceParam wikibase:language "fr,en,[AUTO_LANGUAGE]"
          }
        }
        GROUP BY ?item ?itemLabel ?itemDescription_fr ?itemDescription_en ?itemDescription_any ?parent ?parentLabel
        """,

        "aircraft_models": """
        SELECT DISTINCT ?item 
               ?itemLabel
               (COALESCE(?itemDescription_fr, ?itemDescription_en, ?itemDescription_any, "") AS ?itemDescription)
               ?parent
               ?parentLabel
               (GROUP_CONCAT(DISTINCT ?synonym_fr; separator=" , ") AS ?synonyms_fr)
        WHERE {
          # Tous les modèles d'avions (instances ou sous-classes ou parties)
          ?item wdt:P31/wdt:P279* wd:Q11436 .
          
        
          # On cherche le parent immédiat selon différentes relations         
            OPTIONAL { ?item wdt:P179 ?series .}
            OPTIONAL { ?item wdt:176 ?constructeur.}
            OPTIONAL { ?item wdt:P31 ?instanceOf . } 
            OPTIONAL { ?item wdt:P361 ?partOf . }
            OPTIONAL { ?item wdt:P279 ?subclassOf . }
                       
            BIND(COALESCE(?partOf, ?subclassOf, ?instanceOf, ?series, ?constructeur) AS ?parent)
        
          # Description de l'item (fr > en > autre)
          OPTIONAL { ?item schema:description ?itemDescription_fr . FILTER(LANG(?itemDescription_fr) = "fr") }
          OPTIONAL { ?item schema:description ?itemDescription_en . FILTER(LANG(?itemDescription_en) = "en") }
          OPTIONAL { ?item schema:description ?itemDescription_any .
                     FILTER(LANG(?itemDescription_any) != "fr" && LANG(?itemDescription_any) != "en") }

          # Synonymes en français
          OPTIONAL { ?item skos:altLabel ?synonym_fr . FILTER(LANG(?synonym_fr) = "fr") }
        
          SERVICE wikibase:label {
            bd:serviceParam wikibase:language "fr,en,[AUTO_LANGUAGE]"
          }
        }
        GROUP BY ?item ?itemLabel ?itemDescription_fr ?itemDescription_en ?itemDescription_any ?parent ?parentLabel
        """,

        "aircraft_components": """
        SELECT DISTINCT ?item 
               ?itemLabel
               (COALESCE(?itemDescription_fr, ?itemDescription_en, ?itemDescription_any, "") AS ?itemDescription)
               ?parent
               ?parentLabel
               (GROUP_CONCAT(DISTINCT ?synonym_fr; separator=" , ") AS ?synonyms_fr)
        WHERE {
          # Tous les équipements d'aviation (instances ou sous-classes ou parties)
          ?item wdt:P31*/wdt:P279*/wdt:P361* wd:Q16693356 .
        
          # On cherche le parent immédiat selon différentes relations
            OPTIONAL { ?item wdt:P361 ?partOf . }
            OPTIONAL { ?item wdt:P279 ?subclassOf . }
            OPTIONAL { ?item wdt:P31 ?instanceOf . }
            OPTIONAL { ?item wdt:P452 ?secteur .}
            OPTIONAL { ?item wdt:P176 ?constructeur.}
            
            BIND(COALESCE(?partOf, ?subclassOf, ?instanceOf, ?secteur, ?constructeur) AS ?parent)
        
          # Description de l'item (fr > en > autre)
          OPTIONAL { ?item schema:description ?itemDescription_fr . FILTER(LANG(?itemDescription_fr) = "fr") }
          OPTIONAL { ?item schema:description ?itemDescription_en . FILTER(LANG(?itemDescription_en) = "en") }
          OPTIONAL { ?item schema:description ?itemDescription_any .
                     FILTER(LANG(?itemDescription_any) != "fr" && LANG(?itemDescription_any) != "en") }

          # Synonymes en français
          OPTIONAL { ?item skos:altLabel ?synonym_fr . FILTER(LANG(?synonym_fr) = "fr") }
        
          SERVICE wikibase:label {
            bd:serviceParam wikibase:language "fr,en,[AUTO_LANGUAGE]"
          }
        }
        GROUP BY ?item ?itemLabel ?itemDescription_fr ?itemDescription_en ?itemDescription_any ?parent ?parentLabel
        """,

        "aeronautic_profession": """
        SELECT DISTINCT ?item 
               ?itemLabel
               (COALESCE(?itemDescription_fr, ?itemDescription_en, ?itemDescription_any, "") AS ?itemDescription)
               ?parent
               ?parentLabel
               (GROUP_CONCAT(DISTINCT ?synonym_fr; separator=" , ") AS ?synonyms_fr)
        WHERE {
        ?item wdt:P425* ?domaine.
        VALUES ?domaine { wd:Q765633 wd:Q906438 wd:Q1434048 wd:Q206814 wd:Q627716 wd:Q221395 wd:Q765633 wd:Q22719}.  
        
          # On cherche le parent immédiat selon différentes relations
            OPTIONAL { ?item wdt:P361 ?partOf . }
            OPTIONAL { ?item wdt:P279 ?subclassOf . }
            OPTIONAL { ?item wdt:P31 ?instanceOf . }
            OPTIONAL { ?item wdt:P452 ?secteur .}
            OPTIONAL { ?item wdt:P176 ?constructeur.}
            OPTIONAL { ?item wdt:P749 ?constructeur.}
            
            BIND(COALESCE(?partOf, ?subclassOf, ?instanceOf, ?secteur, ?constructeur, ?domaine) AS ?parent) # Attention à coalesce pour éviter les doublons
        
          # Description de l'item (fr > en > autre)
          OPTIONAL { ?item schema:description ?itemDescription_fr . FILTER(LANG(?itemDescription_fr) = "fr") }
          OPTIONAL { ?item schema:description ?itemDescription_en . FILTER(LANG(?itemDescription_en) = "en") }
          OPTIONAL { ?item schema:description ?itemDescription_any .
                     FILTER(LANG(?itemDescription_any) != "fr" && LANG(?itemDescription_any) != "en") }

          # Synonymes en français
          OPTIONAL { ?item skos:altLabel ?synonym_fr . FILTER(LANG(?synonym_fr) = "fr") }
        
          SERVICE wikibase:label {
            bd:serviceParam wikibase:language "fr,en,[AUTO_LANGUAGE]"
          }
        }
        GROUP BY ?item ?itemLabel ?itemDescription_fr ?itemDescription_en ?itemDescription_any ?parent ?parentLabel
        """
    }
    return queries

print("✅ Requêtes définies (avec synonymes français)")

✅ Requêtes définies (avec synonymes français)


## 🔎 Lancer la recherche

In [1]:
def extract_all_aeronautics_data():
    """Extrait toutes les données aéronautiques de manière optimisée"""
    print("🏗️ EXTRACTION HIÉRARCHIQUE EXHAUSTIVE")
    print("="*50)
    
    queries = build_aeronautics_extraction_queries()
    all_results = []
    
    for category, query in queries.items():
        print(f"\n🔍 Extraction: {category}")
        results = execute_sparql_query(query)
        
        # Enrichir chaque résultat avec sa catégorie
        for result in results:
            result["source_category"] = category
        
        all_results.extend(results)
        print(f"✅ {len(results)} entités trouvées pour {category}")
    
    print(f"\n🎯 TOTAL: {len(all_results)} entités extraites")
    return all_results

raw_aeronautics_data = extract_all_aeronautics_data()

🏗️ EXTRACTION HIÉRARCHIQUE EXHAUSTIVE


NameError: name 'build_aeronautics_extraction_queries' is not defined

### Aperçu

In [6]:
print(raw_aeronautics_data[:5])  # pour afficher un aperçu

json_filename = f"raw.json"
json_filepath = os.path.join(output_dir, json_filename)

with open(json_filepath, 'w', encoding='utf-8') as jsonfile:
    json.dump(raw_aeronautics_data, jsonfile, ensure_ascii=False, indent=2)

NameError: name 'raw_aeronautics_data' is not defined

## 📁 Export

### Construction du fichier

In [ ]:
# 🏗️ CONSTRUCTION DE LA HIÉRARCHIE FINALE

# def get_id_from_uri(uri):
#     # Ex: "http://www.wikidata.org/entity/Q105557" → "Q105557"
#     return uri.split("/")[-1] if uri else ""

def build_final_hierarchy(raw_aeronautics_data):
    """Construit la hiérarchie finale avec parents immédiats et catégories"""
    print("🏗️ CONSTRUCTION DE LA HIÉRARCHIE FINALE")
    print("="*45)
    
    # Créer la hiérarchie structurée
    hierarchy = []
    for entry in raw_aeronautics_data:
    
        hierarchy.append(
            {
            "ID": clean_entity_id(entry.get("item", {}).get("value", "")),
            "Terme": entry.get("itemLabel", {}).get("value", ""),
            "ID_TG": clean_entity_id(entry.get("parent", {}).get("value", "")),
            "TG": entry.get("parentLabel", {}).get("value", ""),
            "Def": entry.get("itemDescription", {}).get("value", ""),
            "EP": entry.get("synonyms_fr", {}).get("value", ""),
            "TA": entry.get("source_category", {})
        }
        )
    
 
    print(f"✅ Hiérarchie construite: {len(hierarchy)} entrées totales")
    return hierarchy

final_thesaurus = build_final_hierarchy(raw_aeronautics_data)
print(f"🎯 Thésaurus final: {len(final_thesaurus)} entrées")

### Export du fichier

In [ ]:
# 💾 EXPORT FINAL UNIQUE - CSV Occidental European Format (semicolon separated)

import os
import csv
import json
from datetime import datetime

def export_final_thesaurus(thesaurus_data):
    """Exporte le thésaurus final en CSV (point-virgule, format européen) et JSON"""
    print("💾 EXPORT FINAL UNIQUE")
    print("="*25)
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # 1. Export CSV - Occidental European (semicolon separator, utf-8-sig BOM)
    csv_filename = f"thesaurus_aeronautique_FINAL_{timestamp}.csv"
    csv_filepath = os.path.join(output_dir, csv_filename)
    
    fieldnames = [
        'ID', 'Terme', 'ID_TG','TG', 'Def', 'EP',
        'TA'
    ]
    
    print(f"📄 Export CSV: {csv_filename}")
    with open(csv_filepath, 'w', newline='', encoding='utf-8-sig') as csvfile:
        writer = csv.DictWriter(
            csvfile, 
            fieldnames=fieldnames,
            delimiter=';',         # Use semicolon as separator
            quoting=csv.QUOTE_MINIMAL
        )
        writer.writeheader()
        for entry in sorted(thesaurus_data, key=lambda x: x["ID"]):
            # Ensure all values are strings and convert None to empty string
            row = {k: ('' if v is None else str(v)) for k, v in entry.items()}
            # Guarantee all required fields exist in row
            for field in fieldnames:
                row.setdefault(field, '')
            writer.writerow(row)
    
    # 2. Export JSON avec métadonnées
    json_filename = f"thesaurus_aeronautique_FINAL_{timestamp}.json"
    json_filepath = os.path.join(output_dir, json_filename)
    
    stats = analyze_thesaurus_statistics(thesaurus_data)
    
    json_data = {
        "metadata": {
            "title": "Thésaurus Aéronautique Final - Wikidata",
            "description": "Thésaurus exhaustif avec hiérarchie et parents immédiats",
            "version": "1.0-FINAL",
            "created": timestamp,
            "source": "Wikidata SPARQL optimisé",
            "total_entries": len(thesaurus_data),
            "extraction_method": "multi-query_hierarchical",
            "parent_detection": "automatic_wikidata_relations",
            "multilingual_support": True,
            "format": "structured_hierarchical_thesaurus"
        },
        "statistics": stats,
        "data": thesaurus_data
    }
    
    print(f"📄 Export JSON: {json_filename}")
    with open(json_filepath, 'w', encoding='utf-8') as jsonfile:
        json.dump(json_data, jsonfile, ensure_ascii=False, indent=2)
    
    return csv_filepath, json_filepath, stats

def analyze_thesaurus_statistics(thesaurus_data):
    """Analyse les statistiques du thésaurus final"""
    stats = {
        "total_entries": len(thesaurus_data),
        "categories": {},
        "relation_types": {},
        "languages": {},
        "hierarchy_depth": 0,
        "entries_with_synonyms": 0,
        "entries_with_descriptions": 0
    }
    
    for entry in thesaurus_data:
        # Catégories
        category = entry.get("category", "unknown")
        stats["categories"][category] = stats["categories"].get(category, 0) + 1
        
        # Types de relation
        rel_type = entry.get("relation_type", "unknown")
        stats["relation_types"][rel_type] = stats["relation_types"].get(rel_type, 0) + 1
        
        # Langues
        lang = entry.get("lang", "unknown")
        stats["languages"][lang] = stats["languages"].get(lang, 0) + 1
        
        # Enrichissements
        if entry.get("synonyms"):
            stats["entries_with_synonyms"] += 1
        if entry.get("description"):
            stats["entries_with_descriptions"] += 1
    
    return stats

def display_final_summary(stats, csv_file, json_file):
    """Affiche un résumé final du thésaurus généré"""
    print("\n🎯 RÉSUMÉ FINAL DU THÉSAURUS AÉRONAUTIQUE")
    print("="*50)
    
    print(f"📊 STATISTIQUES GÉNÉRALES:")
    print(f"   • Total d'entrées: {stats['total_entries']}")
    print(f"   • Entrées avec synonymes: {stats['entries_with_synonyms']}")
    print(f"   • Entrées avec descriptions: {stats['entries_with_descriptions']}")
    
    print(f"\n📂 RÉPARTITION PAR CATÉGORIE:")
    for category, count in sorted(stats["categories"].items(), key=lambda x: x[1], reverse=True):
        percentage = (count / stats['total_entries']) * 100
        print(f"   • {category}: {count} ({percentage:.1f}%)")
    
    print(f"\n🔗 TYPES DE RELATIONS:")
    for rel_type, count in sorted(stats["relation_types"].items(), key=lambda x: x[1], reverse=True):
        print(f"   • {rel_type}: {count}")
    
    print(f"\n🌐 LANGUES:")
    for lang, count in stats["languages"].items():
        print(f"   • {lang}: {count}")
    
    print(f"\n📁 FICHIERS GÉNÉRÉS:")
    print(f"   ✅ CSV: {os.path.basename(csv_file)}")
    print(f"   ✅ JSON: {os.path.basename(json_file)}")
    
    print(f"\n🏆 MISSION ACCOMPLIE !")
    print(f" {stats['total_entries']} entrées de thésaurus")

# Export et résumé final
if final_thesaurus:
    csv_file, json_file, statistics = export_final_thesaurus(final_thesaurus)
    display_final_summary(statistics, csv_file, json_file)
else:
    print("❌ Aucun thésaurus à exporter")

### Nettoyage des doublons

In [ ]:
# Lire le CSV (remplace 'ton_fichier.csv' par le tien)
df = pd.read_csv(csv_file, sep=';', dtype=str).fillna('')

# Fonction pour concaténer les valeurs uniques (séparées par "|")
def concat_unique(series):
    uniques = set([v.strip() for v in series if v.strip() != ''])
    return " | ".join(sorted(uniques)) if uniques else ''

# Grouper par 'ID', en concaténant les valeurs différentes pour chaque colonne
df_clean = df.groupby('ID', as_index=False).agg(concat_unique)

# Sauvegarder le résultat
df_clean.to_csv(csv_file, sep=';', index=False, encoding='utf-8-sig')

print(f"✅ CSV nettoyé et exporté sous {csv_file}")

